# Chapter 2: Basic Analysis
Slicing, Dicing, and Graphing.  Simple exploration of the data.

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join

raw_data_path = '../raw_data/'
logfiles = [f for f in listdir(raw_data_path) if isfile(join(raw_data_path, f))]
print(logfiles)

col_map = {
    'Time (sec)': 'time',
    'A/F Sens 1 Ratio (AFR)': 'afr',
    'AF Sens 1 Ratio (AFR)': 'afr',
    'Boost (PSI)': 'boost',
    'Calculated Load (g/rev)': 'load',
    'Feedback Knock (°)': 'feedback_knock',
    'Fine Knock Learn (°)': 'fine_knock_learn',
    'Gear Position (Gear)': 'gear_position',
    'Ignition Timing (°)': 'timing',
    'MAF (g/s)': 'maf',
    'Primary Ign. (°)': 'primary_timing',
    'RPM (RPM)': 'rpm',
    'TD Burst (%)': 'td_burst',
    'TD Continuous (%)': 'td_continuous',
    'Target Boost (PSI)': 'boost_target',
    'Throttle Pos. (%)': 'throttle',
    'Wastegate Duty (%)': 'wg_duty',
    'Wastegate Max (%)': 'wg_max',
}
dataframes = []

for log in logfiles:
    #print( log )
    # read the logfile
    data = pd.read_csv('../raw_data/{}'.format(log), encoding='iso-8859-1')
    
    #print(data.columns)
    # drop the metadata column:
    data.drop(data.columns[[-1]], axis=1, inplace=True)

    # rename the columns
    data.rename(columns=col_map, inplace=True)
    
    # change delta_t to timestamps
    start_time = pd.Timestamp(log.split('.')[1])
    delta_t = pd.to_timedelta(data['time'], unit='s')
    data['time'] = delta_t + start_time

    # and set the timestamps as the index
    data.index = data.time
    data.drop('time', 1, inplace=True)
                              
    dataframes.append(data)
    
    
# Concatenate all the dataframes together, 
# easier because they are in a list already

data = pd.concat(dataframes)
data.describe()

['roxy_log.201607210800.csv', 'roxy_log.201607211200.csv', 'roxy_log.201607251200.csv', 'roxy_log.201607251500.csv', 'roxy_log.201607250900.csv', 'roxy_log.201607220800.csv']


,afr,boost,load,feedback_knock,fine_knock_learn,gear_position,timing,maf,primary_timing,rpm,td_burst,td_continuous,boost_target,throttle,wg_duty,wg_max
count,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000,109932.000000
mean,15.721728,-7.742465,0.449947,-0.001478,-0.043026,3.243987,25.018784,18.375080,25.298697,2138.596933,-0.002714,4.257299,-4.325508,7.820780,10.800594,13.802727
std,3.368720,4.432594,0.374578,0.055823,0.254251,1.878305,11.863678,25.772601,12.533931,1058.625519,0.177154,1.496412,3.784796,14.451291,14.189460,14.925013
min,11.140000,-12.390000,0.030000,-2.110000,-2.460000,1.000000,-13.500000,1.560000,5.310000,0.000000,-6.000000,-5.030000,-9.750000,0.000000,0.000000,0.000000
25%,14.360000,-10.070000,0.270000,0.000000,0.000000,1.000000,11.500000,4.350000,11.990000,929.000000,0.000000,4.210000,-5.410000,0.000000,0.000000,0.000000
50%,14.590000,-8.910000,0.330000,0.000000,0.000000,3.000000,25.000000,9.930000,26.410000,2336.000000,0.000000,5.000000,-5.340000,4.000000,0.000000,5.610000
75%,14.810000,-7.100000,0.520000,0.000000,0.000000,5.000000,37.000000,22.400000,38.360000,2920.000000,0.000000,5.000000,-5.340000,11.000000,26.670000,31.000000
max,25.730000,18.760000,2.890000,0.000000,0.000000,6.000000,43.500000,245.700000,42.580000,6880.000000,5.000000,5.000000,19.840000,100.000000,45.490000,45.660000


In [4]:
# describe some high-level stats about the data
data.describe()
data.corr()  # show correlation table

,afr,boost,load,feedback_knock,fine_knock_learn,gear_position,timing,maf,primary_timing,rpm,td_burst,td_continuous,boost_target,throttle,wg_duty,wg_max
afr,1.000000,-0.433740,-0.446777,0.028895,0.135342,0.052018,-0.378357,-0.315986,-0.337079,0.085877,0.010449,0.253117,-0.272172,-0.298082,-0.315450,-0.006450
boost,-0.433740,1.000000,0.994464,-0.103424,-0.509755,0.017081,0.061731,0.924141,0.029246,0.326338,-0.056833,-0.575854,0.920081,0.883474,0.480079,0.358789
load,-0.446777,0.994464,1.000000,-0.102388,-0.512309,-0.009865,0.108424,0.934331,0.071267,0.350380,-0.050309,-0.566605,0.917976,0.887826,0.498910,0.362825
feedback_knock,0.028895,-0.103424,-0.102388,1.000000,0.062501,-0.006812,0.026778,-0.092211,0.029227,-0.026600,0.004621,0.016356,-0.126989,-0.138349,-0.045443,-0.042131
fine_knock_learn,0.135342,-0.509755,-0.512309,0.062501,1.000000,-0.000717,0.053697,-0.514840,0.070363,-0.179363,0.021811,0.235729,-0.512309,-0.497853,-0.184569,-0.159564
gear_position,0.052018,0.017081,-0.009865,-0.006812,-0.000717,1.000000,-0.250546,-0.007510,-0.263262,-0.096704,0.008144,0.096623,0.031222,-0.015042,-0.026749,0.030745
timing,-0.378357,0.061731,0.108424,0.026778,0.053697,-0.250546,1.000000,0.159057,0.958698,0.540022,0.021949,-0.198284,-0.064693,0.142223,0.613967,0.436777
maf,-0.315986,0.924141,0.934331,-0.092211,-0.514840,-0.007510,0.159057,1.000000,0.115043,0.558571,-0.047971,-0.459517,0.900011,0.894795,0.578962,0.503651
primary_timing,-0.337079,0.029246,0.071267,0.029227,0.070363,-0.263262,0.958698,0.115043,1.000000,0.532030,0.010083,-0.232674,-0.115353,0.108281,0.586137,0.418292
rpm,0.085877,0.326338,0.350380,-0.026600,-0.179363,-0.096704,0.540022,0.558571,0.532030,1.000000,-0.005242,-0.272152,0.302676,0.453598,0.698714,0.863996


In [5]:
# Covariance
data.cov()

,afr,boost,load,feedback_knock,fine_knock_learn,gear_position,timing,maf,primary_timing,rpm,td_burst,td_continuous,boost_target,throttle,wg_duty,wg_max
afr,11.348273,-6.476674,-0.563765,0.005434,0.115920,0.329140,-15.121208,-27.434126,-14.232589,3.062546e+02,0.006236,1.275963,-3.470168,-14.511308,-15.078608,-0.324293
boost,-6.476674,19.647887,1.651161,-0.025592,-0.574488,0.142209,3.246256,105.573344,1.624847,1.531327e+03,-0.044629,-3.819631,15.435705,56.592434,30.195115,23.736263
load,-0.563765,1.651161,0.140309,-0.002141,-0.048791,-0.006941,0.481822,9.019889,0.334592,1.389391e+02,-0.003338,-0.317595,1.301416,4.805922,2.651739,2.028401
feedback_knock,0.005434,-0.025592,-0.002141,0.003116,0.000887,-0.000714,0.017734,-0.132664,0.020450,-1.571958e+00,0.000046,0.001366,-0.026830,-0.111608,-0.035996,-0.035102
fine_knock_learn,0.115920,-0.574488,-0.048791,0.000887,0.064643,-0.000342,0.161969,-3.373590,0.224229,-4.827677e+01,0.000982,0.089686,-0.492988,-1.829234,-0.665866,-0.605498
gear_position,0.329140,0.142209,-0.006941,-0.000714,-0.000342,3.528029,-5.583060,-0.363529,-6.197850,-1.922892e+02,0.002710,0.271580,0.221955,-0.408292,-0.712911,0.861908
timing,-15.121208,3.246256,0.481822,0.017734,0.161969,-5.583060,140.746866,48.632936,142.557013,6.782237e+03,0.046130,-3.520133,-2.904806,24.383523,103.354747,77.338124
maf,-27.434126,105.573344,9.019889,-0.132664,-3.373590,-0.363529,48.632936,664.226986,37.162446,1.523980e+04,-0.219022,-17.721951,87.790720,333.264087,211.725999,193.732415
primary_timing,-14.232589,1.624847,0.334592,0.020450,0.224229,-6.197850,142.557013,37.162446,157.099426,7.059371e+03,0.022389,-4.364021,-5.472141,19.613156,104.244387,78.249455
rpm,306.254623,1531.326902,138.939123,-1.571958,-48.276772,-192.289181,6782.237275,15239.802558,7059.371256,1.120688e+06,-0.983028,-431.127285,1212.726694,6939.366046,10495.605985,13651.140191


In [7]:
data.loc[data.boost > 7, 'boost', 'wg_duty'].corr()

IndexingError: Too many indexers

In [25]:
# slice out just the boost column
boost = data['boost']
print( boost.head() )
print( boost.describe() )

time
2016-07-21 08:00:00.000   -8.97
2016-07-21 08:00:00.002   -8.97
2016-07-21 08:00:00.003   -9.03
2016-07-21 08:00:00.004   -9.09
2016-07-21 08:00:00.016   -9.05
Name: boost, dtype: float64
count    109932.000000
mean         -7.742465
std           4.432594
min         -12.390000
25%         -10.070000
50%          -8.910000
75%          -7.100000
max          18.760000
Name: boost, dtype: float64


In [26]:
# get just boost above 10psi
boost[boost > 10].count()

1834

In [27]:
# get just boost above 10psi
boost[boost > 10].count()


1834

In [28]:
# look for learned knock events
knock = data['fine_knock_learn']
data[knock < 0].describe()

,afr,boost,load,feedback_knock,fine_knock_learn,gear_position,timing,maf,primary_timing,rpm,td_burst,td_continuous,boost_target,throttle,wg_duty,wg_max
count,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000,3612.000000
mean,13.091603,4.839601,1.519142,-0.019277,-1.309507,3.209856,22.202381,90.769986,20.665291,3227.128184,-0.024983,2.152118,6.394416,45.801495,26.427326,28.178336
std,1.820013,7.447717,0.649686,0.200786,0.555910,1.565069,8.281483,64.310141,10.602877,1099.904999,0.292470,2.146365,7.407231,34.765443,11.120105,10.297728
min,11.140000,-4.760000,0.850000,-2.110000,-2.460000,1.000000,-2.500000,17.840000,5.660000,1547.000000,-5.030000,-5.030000,-8.160000,0.000000,0.000000,0.000000
25%,11.250000,-1.280000,0.990000,0.000000,-1.760000,2.000000,14.000000,39.770000,9.880000,2347.000000,0.000000,0.600000,0.330000,20.000000,18.820000,21.665000
50%,13.440000,0.830000,1.165000,0.000000,-1.050000,3.000000,22.500000,53.030000,20.080000,2719.500000,0.000000,1.800000,3.270000,27.000000,25.880000,26.895000
75%,14.360000,11.270000,1.980000,0.000000,-1.050000,4.000000,30.000000,136.510000,32.030000,4194.000000,0.000000,4.480000,15.410000,100.000000,34.510000,35.640000
max,25.730000,18.760000,2.890000,0.000000,-0.350000,6.000000,40.000000,245.700000,42.580000,6769.000000,1.230000,5.000000,19.840000,100.000000,45.490000,45.660000


### Filter down to just OverBoost Events

In [29]:
# return a filterd dataframe, containing only samples where boost > boost_target
overboost = data[data.boost > data.boost_target]
overboost.describe()

,afr,boost,load,feedback_knock,fine_knock_learn,gear_position,timing,maf,primary_timing,rpm,td_burst,td_continuous,boost_target,throttle,wg_duty,wg_max
count,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000,3544.000000
mean,13.321318,1.159797,1.174746,-0.007144,-0.262960,2.925790,22.319695,57.736024,24.247046,2641.771445,-0.198183,-0.992861,0.015418,24.251129,14.214328,20.172438
std,1.744837,7.353299,0.638637,0.122589,0.593829,1.854995,9.908775,50.824361,11.407939,1156.803422,0.811334,1.641017,7.435647,26.352034,15.676016,16.932274
min,11.140000,-8.970000,0.140000,-2.110000,-2.460000,1.000000,-13.500000,1.650000,5.660000,0.000000,-6.000000,-5.030000,-9.750000,0.000000,0.000000,0.000000
25%,11.250000,-4.080000,0.740000,0.000000,0.000000,1.000000,14.500000,21.047500,11.990000,1619.750000,0.000000,-1.630000,-5.320000,11.000000,0.000000,0.000000
50%,13.780000,-2.110000,0.880000,0.000000,0.000000,3.000000,22.000000,32.570000,26.760000,2759.500000,0.000000,-0.480000,-3.330000,14.000000,5.880000,26.560000
75%,14.470000,5.470000,1.500000,0.000000,0.000000,4.000000,30.500000,81.432500,33.790000,3606.000000,0.000000,0.000000,4.160000,26.000000,29.020000,35.250000
max,24.810000,18.760000,2.890000,0.000000,0.000000,6.000000,42.000000,238.250000,42.580000,6769.000000,1.550000,5.000000,17.980000,100.000000,45.490000,45.660000


In [30]:
# get just the boost value, where boost > target
data[data.boost > data.boost_target]['boost'].describe()
# Best practice is to use loc method:
data.loc[data.boost > data.boost_target, 'boost'].describe()

count    3544.000000
mean        1.159797
std         7.353299
min        -8.970000
25%        -4.080000
50%        -2.110000
75%         5.470000
max        18.760000
Name: boost, dtype: float64

NOTE: looks like almost all of our overboost events are where boost is negative.

Probably should adjust the map so that we target more reasonable boost levels for these areas.

consider setting minimum boost targets to 0psi, and adjusting the max WGDC so that it is the MINIMUM WGDC needed to hit peak boost.
Or, perhaps, because the WG will only be able to open above ~7psi [WG rod begins to aggressively move at ~7psi](https://cobbtuning.zendesk.com/hc/en-us/articles/200025164-How-Subaru-s-Factory-Boost-Control-System-Works), set the minimum targets to that value, WGDC will be irrelevant below the 7psi threshold.

In [31]:
# filter rows based on multiple criteria:
# use &, |, and parens for evaluation order.
data[(data.boost > 0) & (data.boost > data.boost_target)].describe()

,afr,boost,load,feedback_knock,fine_knock_learn,gear_position,timing,maf,primary_timing,rpm,td_burst,td_continuous,boost_target,throttle,wg_duty,wg_max
count,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000
mean,11.981283,9.349699,1.872048,-0.013045,-0.551662,4.372488,16.365533,112.257859,12.199057,3546.671561,-0.127550,-1.101298,8.083006,45.703246,28.367156,36.120139
std,1.211161,6.108758,0.559538,0.165456,0.741325,1.306910,4.862701,46.270884,6.104407,659.135396,0.582461,1.657802,6.592980,31.503913,12.326410,5.990177
min,11.140000,0.020000,0.480000,-2.110000,-2.460000,1.000000,4.500000,19.120000,5.660000,795.000000,-5.030000,-5.030000,-8.160000,0.000000,0.000000,1.340000
25%,11.250000,4.270000,1.422500,0.000000,-1.050000,4.000000,12.000000,75.830000,7.770000,3120.500000,0.000000,-1.630000,2.590000,24.000000,19.317500,32.165000
50%,11.250000,8.880000,1.780000,0.000000,0.000000,4.000000,15.500000,106.600000,10.590000,3571.000000,0.000000,-0.380000,7.520000,31.000000,29.410000,35.430000
75%,12.400000,16.695000,2.500000,0.000000,0.000000,6.000000,19.000000,146.285000,14.450000,3983.000000,0.000000,0.000000,13.560000,60.000000,36.470000,37.490000
max,21.360000,18.760000,2.890000,0.000000,0.000000,6.000000,40.000000,238.250000,42.580000,5756.000000,0.970000,5.000000,17.980000,100.000000,45.490000,45.660000


In [10]:
ob = data[(data.boost > 7) & (data.boost > data.boost_target)]

ob.describe()

,afr,boost,load,feedback_knock,fine_knock_learn,gear_position,timing,maf,primary_timing,rpm,td_burst,td_continuous,boost_target,throttle,wg_duty,wg_max
count,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000
mean,11.247371,13.969133,2.288587,-0.023608,-0.804168,4.639161,13.092308,145.108811,8.538266,3834.974825,-0.116476,-0.911427,12.833930,63.479720,35.363678,39.485049
std,0.040284,4.002584,0.386794,0.222093,0.791301,1.316191,2.217169,33.939852,2.829764,473.349504,0.514574,1.578267,4.620334,31.434705,9.221620,5.194614
min,11.140000,7.040000,1.200000,-2.110000,-2.460000,1.000000,9.500000,69.590000,5.660000,2518.000000,-5.030000,-5.030000,-8.160000,0.000000,1.570000,22.390000
25%,11.250000,9.730000,1.890000,0.000000,-1.410000,4.000000,11.500000,119.875000,6.720000,3512.000000,0.000000,-1.320000,8.970000,35.000000,29.410000,36.080000
50%,11.250000,14.310000,2.320000,0.000000,-0.700000,4.000000,13.000000,139.060000,8.120000,3848.000000,0.000000,-0.210000,12.340000,48.000000,34.900000,37.370000
75%,11.250000,18.000000,2.660000,0.000000,0.000000,6.000000,15.000000,176.810000,9.880000,4167.500000,0.000000,0.000000,17.730000,100.000000,44.710000,45.660000
max,11.830000,18.760000,2.890000,0.000000,0.000000,6.000000,28.500000,238.250000,42.580000,5756.000000,0.970000,5.000000,17.980000,100.000000,45.490000,45.660000


In [12]:
ob = data[(data.boost > 7) & (data.boost < data.boost_target)]

ob.cov()

,afr,boost,load,feedback_knock,fine_knock_learn,gear_position,timing,maf,primary_timing,rpm,td_burst,td_continuous,boost_target,throttle,wg_duty,wg_max
afr,0.018735,-0.134749,-0.012100,0.001205,0.011661,-0.001346,0.052782,-1.167467,0.034973,-13.984494,-0.000098,0.032900,-0.032933,-0.191260,0.062730,0.066180
boost,-0.134749,9.173321,0.810425,0.068959,-0.732193,0.107262,-3.355299,78.887654,-2.112132,872.184820,-0.019147,-2.206964,5.596125,33.484025,7.437323,5.635224
load,-0.012100,0.810425,0.077126,0.007012,-0.079986,-0.006063,-0.253316,8.054710,-0.172176,94.133904,-0.001631,-0.192141,0.506779,2.919388,0.686629,0.520083
feedback_knock,0.001205,0.068959,0.007012,0.118233,-0.015646,0.003723,0.173056,1.921754,0.061762,47.762296,0.000269,-0.044722,0.003196,-0.390610,-0.283960,-0.270555
fine_knock_learn,0.011661,-0.732193,-0.079986,-0.015646,0.656729,0.166736,0.196051,-20.454064,-0.362938,-380.110544,0.000116,0.115906,-0.495376,-4.027213,1.888752,2.096240
gear_position,-0.001346,0.107262,-0.006063,0.003723,0.166736,1.933343,-0.499389,-20.362313,-0.935830,-513.247414,-0.003228,-0.363284,-0.134045,0.495684,2.142773,2.399834
timing,0.052782,-3.355299,-0.253316,0.173056,0.196051,-0.499389,3.932838,14.177988,2.642601,764.215441,0.011915,0.648832,-2.501749,-19.357862,-10.079871,-9.341642
maf,-1.167467,78.887654,8.054710,1.921754,-20.454064,-20.362313,14.177988,2043.068680,38.653340,40112.187202,-0.178994,-8.378719,53.611012,238.826046,-128.654659,-151.172329
primary_timing,0.034973,-2.112132,-0.172176,0.061762,-0.362938,-0.935830,2.642601,38.653340,3.554928,1259.754270,0.008148,0.537315,-1.518003,-15.649758,-11.089089,-10.666975
rpm,-13.984494,872.184820,94.133904,47.762296,-380.110544,-513.247414,764.215441,40112.187202,1259.754270,899303.206210,-0.765315,-14.540518,560.905983,1092.607517,-4478.744006,-4752.408509
